In [1]:
!pip install -q openai

In [2]:
import os
import json
import asyncio
import random
from openai import OpenAI, AsyncOpenAI
from google.colab import userdata


# Initialize the OpenAI clients (sync and async)
# The clients automatically read the OPENAI_API_KEY from the environment
try:
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    aclient = AsyncOpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    print("OpenAI clients initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    print("Please ensure your OPENAI_API_KEY is set correctly in Colab.")

OpenAI clients initialized successfully.


# The problem with a single, large LLM call

In [3]:
# Here we define our mock webpage content. Each source has a title and text.

webpage_1 = {
    "title": "The Benefits of Solar Energy",
    "content": """
    Solar energy is a renewable powerhouse, offering numerous environmental and economic benefits.
    By converting sunlight into electricity through photovoltaic (PV) panels, it reduces reliance on fossil fuels,
    thereby cutting down greenhouse gas emissions. Homeowners who install solar panels can significantly
    lower their monthly electricity bills, and in some cases, sell excess power back to the grid.
    While the initial installation cost can be high, government incentives and long-term savings make
    it a financially viable option for many. Solar power is also a key component in achieving energy
    independence for nations worldwide.
    """
}

webpage_2 = {
    "title": "Understanding Wind Turbines",
    "content": """
    Wind turbines are towering structures that capture kinetic energy from the wind and convert it into
    electrical power. They are a critical part of the global shift towards sustainable energy.
    Turbines can be installed both onshore and offshore, with offshore wind farms generally producing more
    consistent power due to stronger, more reliable winds. The main challenge for wind energy is its
    intermittency—it only generates power when the wind blows. This necessitates the use of energy
    storage solutions, like large-scale batteries, to ensure a steady supply of electricity.
    """
}

webpage_3 = {
    "title": "Energy Storage Solutions",
    "content": """
    Effective energy storage is the key to unlocking the full potential of renewable sources like solar
    and wind. Because these sources are intermittent, storing excess energy when it's plentiful and
    releasing it when it's needed is crucial for a stable power grid. The most common form of
    large-scale storage is pumped-hydro storage, but battery technologies, particularly lithium-ion,
    are rapidly becoming more affordable and widespread. These batteries can be used in homes, businesses,
    and at the utility scale to balance energy supply and demand, making our energy system more
    resilient and reliable.
    """
}

all_sources = [webpage_1, webpage_2, webpage_3]

# We'll combine the content for the LLM to process
combined_content = "\n\n".join([f"Source Title: {source['title']}\nContent: {source['content']}" for source in all_sources])

In [4]:
# This prompt tries to do everything at once: generate questions, find answers,
# and cite sources. This complexity can often confuse the model.
prompt_complex = f"""
Based on the provided content from three webpages, generate a list of exactly 10 frequently asked questions (FAQs).
For each question, provide a concise answer derived ONLY from the text.
After each answer, you MUST include a list of the 'Source Title's that were used to formulate that answer.

Your final output should be a JSON array where each object has three keys: "question", "answer", and "sources" (which is an array of strings).

Provided Content:
---
{combined_content}
---
""".strip()

response_complex = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[
        {"role": "system", "content": "You are an expert at creating FAQs from provided documents."},
        {"role": "user", "content": prompt_complex}
    ],
    response_format={"type": "json_object"}
)
# Note: Even with JSON mode, the model might fail to follow all instructions perfectly.
result_complex = response_complex.choices[0].message.content
print("Complex prompt result (might be inconsistent):")
print(json.dumps(json.loads(result_complex), indent=2))

Complex prompt result (might be inconsistent):
{
  "faqs": [
    {
      "question": "What are the main benefits of solar energy?",
      "answer": "Solar energy reduces reliance on fossil fuels, cuts greenhouse gas emissions, lowers electricity bills, and can allow homeowners to sell excess power back to the grid. It is also a key component in achieving energy independence for nations.",
      "sources": [
        "The Benefits of Solar Energy"
      ]
    },
    {
      "question": "How do wind turbines generate electricity?",
      "answer": "Wind turbines capture kinetic energy from the wind and convert it into electrical power, functioning as a critical part of sustainable energy systems.",
      "sources": [
        "Understanding Wind Turbines"
      ]
    },
    {
      "question": "Where can wind turbines be installed?",
      "answer": "Wind turbines can be installed both onshore and offshore, with offshore turbines generally producing more consistent power due to stronger, m

# Building a sequential workflow

In [5]:
# Prompts
prompt_generate_questions = """
Based on the content below, generate a list of 10 relevant and distinct questions that a user might have.
Return these questions as a JSON array of strings.

Provided Content:
---
{combined_content}
---
""".strip()

prompt_answer_question = """
Using ONLY the provided content below, answer the following question.
The answer should be concise and directly address the question.

Question:
"{question}"

Provided Content:
---
{combined_content}
---
""".strip()

prompt_find_sources = """
You will be given a question and an answer that was generated from a set of documents.
Your task is to identify which of the original documents were used to create the answer.
Return a JSON object with a single key "sources" which is a list of the titles of the relevant documents.

Question: "{question}"
Answer: "{answer}"

Documents:
---
{combined_content}
---
""".strip()

In [6]:
# Step 1: Generate questions
response_questions = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates questions based on text."},
        {"role": "user", "content": prompt_generate_questions.format(combined_content=combined_content)}
    ],
    response_format={"type": "json_object"}
)

generated_questions = json.loads(response_questions.choices[0].message.content)['questions']
print(f"Successfully generated {len(generated_questions)} questions.")

# Steps 2 & 3: Answer and find sources for each question
final_faqs = []
for question in generated_questions:
    print(f"  - Processing: '{question[:50]}...'")

    # Step 2: Generate an answer for the current question
    prompt_answer_question = f"""
    Using ONLY the provided content below, answer the following question.
    The answer should be concise and directly address the question.

    Question:
    "{question}"

    Provided Content:
    ---
    {combined_content}
    ---
    """
    answer_response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": "You answer questions based *only* on the provided context."},
            {"role": "user", "content": prompt_answer_question.format(combined_content=combined_content)}
        ]
    )
    answer = answer_response.choices[0].message.content

    # Step 3: Identify the sources for the generated answer
    sources_response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": "You are an expert at identifying document sources for a given text."},
            {"role": "user", "content": prompt_find_sources.format(question=question, answer=answer, combined_content=combined_content)}
        ],
        response_format={"type": "json_object"}
    )
    sources = json.loads(sources_response.choices[0].message.content)['sources']

    final_faqs.append({
        "question": question,
        "answer": answer,
        "sources": sources
    })

# Final result
print("\nGenerated FAQ List:")
print(json.dumps(final_faqs, indent=2))

Successfully generated 10 questions.
  - Processing: 'What are the main environmental benefits of using ...'
  - Processing: 'How do photovoltaic (PV) panels convert sunlight i...'
  - Processing: 'What financial incentives are available for homeow...'
  - Processing: 'How does wind energy contribute to sustainable ene...'
  - Processing: 'What are the differences between onshore and offsh...'
  - Processing: 'Why is wind energy considered intermittent, and ho...'
  - Processing: 'What role do energy storage solutions play in rene...'
  - Processing: 'How do batteries like lithium-ion enhance the stab...'
  - Processing: 'What are the advantages and disadvantages of pumpe...'
  - Processing: 'What challenges are associated with integrating re...'

Generated FAQ List:
[
  {
    "question": "What are the main environmental benefits of using solar energy?",
    "answer": "The main environmental benefits of using solar energy are reducing reliance on fossil fuels and cutting down greenhous

# Building a routing workflow

In [7]:
prompt_classification = """
Classify the user's query into one of the following categories:
"Technical Support", "Billing Inquiry", or "General Question".
Return only the category name and nothing else.
User Query: "{user_query}"
""".strip()

def classify_intent(user_query):
    """Uses an LLM to classify a user query."""
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are an expert at classifying user intents."},
            {"role": "user", "content": prompt_classification.format(user_query=user_query)}
        ]
    )
    intent = response.choices[0].message.content.strip()
    return intent

query_1 = "My internet connection is not working."
query_2 = "I think there is a mistake on my last invoice."
query_3 = "What are your opening hours?"

intent_1 = classify_intent(query_1)
print(f"Query: {query_1}\nIntent: {intent_1}\n")
intent_2 = classify_intent(query_2)
print(f"Query: {query_2}\nIntent: {intent_2}\n")
intent_3 = classify_intent(query_3)
print(f"Query: {query_3}\nIntent: {intent_3}\n")

Query: My internet connection is not working.
Intent: Technical Support

Query: I think there is a mistake on my last invoice.
Intent: Billing Inquiry

Query: What are your opening hours?
Intent: General Question



In [8]:
prompt_technical_support = """
You are a helpful technical support agent.
The user says: '{user_query}'.
Provide a helpful first response, asking for more details like what troubleshooting steps they have already tried.
""".strip()

prompt_billing_inquiry = """
You are a helpful billing support agent.
The user says: '{user_query}'.
Acknowledge their concern and inform them that you will need to look up their account, asking for their account number.
""".strip()

prompt_general_question = """
You are a general assistant.
The user says: '{user_query}'.
Apologize that you are not sure how to help and ask them to rephrase their question.
""".strip()

prompt_general_question = """
You are a general assistant.
The user says: '{user_query}'.
Apologize that you are not sure how to help and ask them to rephrase their question.
""".strip()

def handle_query(user_query, intent):
    """Routes a query to the correct handler based on its classified intent."""
    if intent == "Technical Support":
        prompt = prompt_technical_support.format(user_query=user_query)
    elif intent == "Billing Inquiry":
        prompt = prompt_billing_inquiry.format(user_query=user_query)
    elif intent == "General Question":
        prompt = prompt_general_question.format(user_query=user_query)
    else:
        prompt = prompt_general_question.format(user_query=user_query)
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}]
    )
    return response.choices[0].message.content

response_1 = handle_query(query_1, intent_1)
print(f"Query: {query_1}\nIntent: {intent_1}\nResponse: {response_1}\n")

response_2 = handle_query(query_2, intent_2)
print(f"Query: {query_2}\nIntent: {intent_2}\nResponse: {response_2}\n")

response_3 = handle_query(query_3, intent_3)
print(f"Query: {query_3}\nIntent: {intent_3}\nResponse: {response_3}\n")

Query: My internet connection is not working.
Intent: Technical Support
Response: I'm sorry to hear that your internet connection isn't working. Could you please provide a bit more detail? For example, are you using a wired or wireless connection? Have you tried any troubleshooting steps so far, such as restarting your modem/router or checking if other devices can connect? This info will help me assist you better.

Query: I think there is a mistake on my last invoice.
Intent: Billing Inquiry
Response: I'm sorry to hear that you think there may be a mistake on your last invoice. To assist you further, could you please provide me with your account number? That way, I can look up your account and review the details.

Query: What are your opening hours?
Intent: General Question
Response: I'm sorry, I'm not sure how to help with that. Could you please rephrase your question or provide more details?



# Orchestrator-worker pattern

In [9]:
# Orchestrator
prompt_orchestrator = """
You are a master orchestrator. Your job is to break down a complex user query into a JSON array of objects.
Each object represents one sub-task and must have a "query_type" and relevant parameters.

The possible "query_type" values are:
1. "BillingInquiry": Requires "invoice_number".
2. "ProductReturn": Requires "product_name" and "reason_for_return".
3. "StatusUpdate": Requires "order_id".

User Query:
---
{complex_query}
---

Return ONLY the JSON array and nothing else. Start with "[", end with "]" and separate objects with a comma.
""".strip()

async def orchestrator(complex_query):
    """Breaks down a complex query into a list of tasks."""
    response = await aclient.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt_orchestrator.format(complex_query=complex_query)}]
    )
    tasks_str = response.choices[0].message.content
    try:
        return json.loads(tasks_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from orchestrator: {e}")
        print(f"Orchestrator raw response: {tasks_str}")
        return []

In [10]:
# Worker for Billing Inquiry
prompt_billing_worker_extractor = """
You are a specialized assistant. A user has a query regarding invoice '{invoice_number}'.
From the full user query provided below, extract the specific concern or question the user has voiced about this particular invoice.
Respond with ONLY the extracted concern/question. If no specific concern is mentioned beyond a general inquiry about the invoice, state 'General inquiry regarding the invoice'.

Full User Query:
---
{original_user_query}
---

Extracted concern about invoice {invoice_number}:
""".strip()

async def handle_billing_worker(invoice_number, original_user_query):
    """
    Handles a billing inquiry.
    1. Uses an LLM to extract the specific concern about the invoice from the original query.
    2. Simulates opening an investigation.
    3. Returns structured data about the action taken.
    """
    extraction_prompt = prompt_billing_worker_extractor.format(
        invoice_number=invoice_number,
        original_user_query=original_user_query
    )
    response = await aclient.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": extraction_prompt}]
    )
    extracted_concern = response.choices[0].message.content.strip()

    # Simulate backend action: opening an investigation
    print(f"  [Billing Worker] Action: Investigating invoice {invoice_number} for concern: '{extracted_concern}'")
    investigation_id = f"INV_CASE_{random.randint(1000,9999)}"
    eta_days = 2

    return {
        "task": "Billing Inquiry",
        "invoice_number": invoice_number,
        "user_concern": extracted_concern,
        "action_taken": f"An investigation (Case ID: {investigation_id}) has been opened regarding your concern.",
        "resolution_eta": f"{eta_days} business days"
    }

In [11]:
# Worker for Product Return
async def handle_return_worker(product_name, reason_for_return):
    """
    Handles a product return request.
    1. Simulates generating an RMA number and providing return instructions.
    2. Returns structured data.
    """
    # Simulate backend action: generating RMA and getting instructions
    rma_number = f"RMA-{random.randint(10000, 99999)}"
    shipping_instructions = (
        "Please pack the '{product_name}' securely in its original packaging if possible. "
        "Include all accessories and manuals. Write the RMA number ({rma_number}) clearly on the outside of the package. "
        "Ship to: Returns Department, 123 Automation Lane, Tech City, TC 98765."
    ).format(product_name=product_name, rma_number=rma_number)
    print(f"  [Return Worker] Action: Generated RMA {rma_number} for {product_name} (Reason: {reason_for_return})")

    return {
        "task": "Product Return",
        "product_name": product_name,
        "reason_for_return": reason_for_return,
        "rma_number": rma_number,
        "shipping_instructions": shipping_instructions
    }

In [12]:
# Worker for Status Update
async def handle_status_worker(order_id):
    """
    Handles an order status update request.
    1. Simulates fetching order status from a backend system.
    2. Returns structured data.
    """
    # Simulate backend action: fetching order status
    # Possible statuses and details to make it more dynamic
    possible_statuses = [
        {"status": "Processing", "carrier": "N/A", "tracking": "N/A", "delivery_estimate": "3-5 business days"},
        {"status": "Shipped", "carrier": "SuperFast Shipping", "tracking": f"SF{random.randint(100000,999999)}", "delivery_estimate": "Tomorrow"},
        {"status": "Delivered", "carrier": "Local Courier", "tracking": f"LC{random.randint(10000,99999)}", "delivery_estimate": "Delivered yesterday"},
        {"status": "Delayed", "carrier": "Standard Post", "tracking": f"SP{random.randint(10000,99999)}", "delivery_estimate": "Expected in 2-3 additional days"}
    ]
    # For a given order_id, we could hash it to pick a status or just pick one randomly for this example
    # This ensures that for the same order_id in a single run, we'd get the same fake status if we implement a simple hash.
    # For now, let's pick randomly for demonstration.
    status_details = random.choice(possible_statuses)
    print(f"  [Status Worker] Action: Fetched status for order {order_id}: {status_details['status']}")

    return {
        "task": "Status Update",
        "order_id": order_id,
        "current_status": status_details["status"],
        "carrier": status_details["carrier"],
        "tracking_number": status_details["tracking"],
        "expected_delivery": status_details["delivery_estimate"]
    }

In [13]:
# Synthesizer
prompt_synthesizer = """
You are a master communicator. Combine several distinct pieces of information from our support team into a single, well-formatted, and friendly email to a customer.

Here are the points to include, based on the actions taken for their query:
---
{formatted_results}
---

Combine these points into one cohesive response. Start with a friendly greeting (e.g., "Dear Customer," or "Hi there,") and end with a polite closing (e.g., "Sincerely," or "Best regards,").
Ensure the tone is helpful and professional.
""".strip()

async def synthesizer(results):
    """Combines structured results from workers into a single user-facing message."""
    bullet_points = []
    for res in results:
        point = f"Regarding your {res['task']}:\n"
        if res['task'] == 'Billing Inquiry':
            point += f"  - Invoice Number: {res['invoice_number']}\n"
            point += f"  - Your Stated Concern: \"{res['user_concern']}\"\n"
            point += f"  - Our Action: {res['action_taken']}\n"
            point += f"  - Expected Resolution: We will get back to you within {res['resolution_eta']}."
        elif res['task'] == 'ProductReturn':
            point += f"  - Product: {res['product_name']}\n"
            point += f"  - Reason for Return: \"{res['reason_for_return']}\"\n"
            point += f"  - Return Authorization (RMA): {res['rma_number']}\n"
            point += f"  - Instructions: {res['shipping_instructions']}"
        elif res['task'] == 'Status Update':
            point += f"  - Order ID: {res['order_id']}\n"
            point += f"  - Current Status: {res['current_status']}\n"
            if res['carrier'] != "N/A":
                point += f"  - Carrier: {res['carrier']}\n"
            if res['tracking_number'] != "N/A":
                point += f"  - Tracking Number: {res['tracking_number']}\n"
            point += f"  - Delivery Estimate: {res['expected_delivery']}"
        bullet_points.append(point)

    formatted_results = "\n\n".join(bullet_points)
    prompt = prompt_synthesizer.format(formatted_results=formatted_results)
    response = await aclient.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [14]:
# Test with customer query
complex_customer_query = """
Hi, I'm writing to you because I have a question about invoice #INV-7890. It seems higher than I expected.
Also, I would like to return the 'SuperWidget 5000' I bought because it's not compatible with my system.
Finally, can you give me an update on my order #A-12345?
""".strip()

async def process_user_query(user_query):
    """Processes a query using the Orchestrator-Worker-Synthesizer pattern."""

    print(f"User query:\n---\n{user_query}\n---")

    # 1. Run orchestrator
    tasks_list = await orchestrator(user_query)
    if not tasks_list:
        print("\nOrchestrator did not return any tasks. Exiting.")
        return

    print("\nDeconstructed tasks from Orchestrator:")
    print(json.dumps(tasks_list, indent=2))

    # 2. Run workers
    worker_results = []
    if tasks_list:
        worker_coroutines = []
        print(f"\nDispatching {len(tasks_list)} workers...")
        for task in tasks_list:
            if task['query_type'] == 'BillingInquiry':
                worker_coroutines.append(handle_billing_worker(task['invoice_number'], user_query))
            elif task['query_type'] == 'ProductReturn':
                # Ensure reason_for_return is present, provide a default if not (though orchestrator should capture it)
                reason = task.get('reason_for_return', 'Not specified by user')
                worker_coroutines.append(handle_return_worker(task['product_name'], reason))
            elif task['query_type'] == 'StatusUpdate':
                worker_coroutines.append(handle_status_worker(task['order_id']))
            else:
                print(f"Warning: Unknown query_type '{task.get('query_type')}' found in orchestrator tasks.")

        if worker_coroutines:
            print(f"Running {len(worker_coroutines)} workers concurrently...")
            worker_results = await asyncio.gather(*worker_coroutines)
            print("\nWorkers finished their jobs. Results:")
            for i, res in enumerate(worker_results):
                print(f"--- Worker Result {i+1} ---")
                print(json.dumps(res, indent=2))
                print("----------------------")
        else:
            print("\nNo valid worker tasks to run.")
    else:
        print("\nNo tasks to run for workers.")

    # 3. Run synthesizer
    if worker_results:
        print("\nSynthesizing final response...")
        final_user_message = await synthesizer(worker_results)
        print("\n--- Final Synthesized Response ---")
        print(final_user_message)
        print("---------------------------------")
    else:
        print("\nSkipping synthesis because there were no worker results.")

await process_user_query(complex_customer_query)

User query:
---
Hi, I'm writing to you because I have a question about invoice #INV-7890. It seems higher than I expected.
Also, I would like to return the 'SuperWidget 5000' I bought because it's not compatible with my system.
Finally, can you give me an update on my order #A-12345?
---

Deconstructed tasks from Orchestrator:
[
  {
    "query_type": "BillingInquiry",
    "invoice_number": "INV-7890"
  },
  {
    "query_type": "ProductReturn",
    "product_name": "SuperWidget 5000",
    "reason_for_return": "not compatible with my system"
  },
  {
    "query_type": "StatusUpdate",
    "order_id": "A-12345"
  }
]

Dispatching 3 workers...
Running 3 workers concurrently...
  [Return Worker] Action: Generated RMA RMA-63742 for SuperWidget 5000 (Reason: not compatible with my system)
  [Status Worker] Action: Fetched status for order A-12345: Delayed
  [Billing Worker] Action: Investigating invoice INV-7890 for concern: 'The invoice amount seems higher than expected.'

Workers finished the